<a href="https://colab.research.google.com/github/ivanshauck/measuring-offensive-baseball-value/blob/main/baseball_ML_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this portion of my project I will create a linear regression that relates offensive outcomes to runs scored, I will do this with a few different data sets, for different purposes. However the main purpose is to find linear weights for different offensive outcomes. I want to know how valuable a single is compared to a walk, how valuable a double is compared to a triple, and so on. The goal is to ultimately have runs as a function of the offensive outcomes, with statistically significant linear weights produced from the regression. Once I have this equation, I believe I will have the basis for a formula that can measure a player's offensive production, in a season, or a career, or just a game, based on his game logs.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('/content/baseball.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123270 entries, 0 to 123269
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       123270 non-null  int64  
 1   Unnamed: 0.1     123270 non-null  int64  
 2   year             123270 non-null  int64  
 3   team             123270 non-null  object 
 4   score            123270 non-null  int64  
 5   at_bats          123270 non-null  float64
 6   hits             123270 non-null  float64
 7   doubles          123270 non-null  float64
 8   triples          123270 non-null  float64
 9   home_runs        123270 non-null  float64
 10  rbi              123270 non-null  float64
 11  sacrifice_hits   123270 non-null  float64
 12  sacrifice_flies  123270 non-null  float64
 13  walks            123270 non-null  float64
 14  stolen_bases     123270 non-null  float64
 15  home/away        123270 non-null  object 
 16  singles          123270 non-null  floa

This will be the base regression, here I'm just using the offensive outcomes I'm interested in looking at. These are singles, doubles, triples, home runs, walks, stolen bases, and sacrifice flies. I understand that how often one strikes out, or how often one grounds into a double play also affects their offensive value. However for this project I mainly just want to look at positive outcomes, to measure the aggregate of all positive outcomes a player produces.

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm

X = data[['singles','doubles','triples','home_runs','walks','stolen_bases','sacrifice_flies']]
Y = data['score']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=33)
X_train = sm.add_constant(X_train)
results = sm.OLS(Y_train, X_train).fit()
print(results.summary())



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                 3.902e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):               0.00
Time:                        15:30:23   Log-Likelihood:            -1.8574e+05
No. Observations:               98616   AIC:                         3.715e+05
Df Residuals:                   98608   BIC:                         3.716e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.3193      0.015   -1

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
lrm = linear_model.LinearRegression()
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=33)
lrm.fit(X_train,Y_train)
ytrain_pred = lrm.predict(X_train)
Ytest_pred = lrm.predict(X_test)
print('R squared score for training set: {}'.format(lrm.score(X_train,Y_train)))
print('R squared score for test set: {}'.format(lrm.score(X_test,Y_test)))
print('cross validation scores on training set: {}'.format(cross_val_score(lrm,X_train,Y_train,cv=5)))
print('mean squared error on the test set: {}'.format(mean_squared_error(Y_test,Ytest_pred)))
print('mean absolute error on the test set: {}'.format(mean_absolute_error(Y_test,Ytest_pred)))
print('median absolute error on the test set: {}'.format(median_absolute_error(Y_test,Ytest_pred)))


R squared score for training set: 0.7347209080778049
R squared score for test set: 0.731165268317264
cross validation scores on training set: [0.73596322 0.73384743 0.73909245 0.73266319 0.73182725]
mean squared error on the test set: 2.527562062312236
mean absolute error on the test set: 1.2321541863642704
median absolute error on the test set: 1.0002564462344559


So for the initial regression we get an R squared of .735, this tells us that the variation in our independent variables accounts for about 73-74% of the variation in runs scored. Moreover all coefficients have a p value well below .01, and can be deemed statistically significant at a level of .01 or below, which is good. We also have a mean absolute error of 1.23, so on average this model erres by 1.23 runs. However the median absolute error is about 1 run even.

Below is the regression run on the winsorized dataset, this has a much lower R squared at .67, I'm going to leave this one alone. 

In [ ]:
win_data = pd.read_csv('baseball_winsorized.csv')

In [ ]:
win_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123270 entries, 0 to 123269
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       123270 non-null  int64  
 1   Unnamed: 0.1     123270 non-null  int64  
 2   year             123270 non-null  int64  
 3   team             123270 non-null  object 
 4   score            123270 non-null  int64  
 5   at_bats          123270 non-null  float64
 6   hits             123270 non-null  float64
 7   doubles          123270 non-null  float64
 8   triples          123270 non-null  float64
 9   home_runs        123270 non-null  float64
 10  rbi              123270 non-null  float64
 11  sacrifice_hits   123270 non-null  float64
 12  sacrifice_flies  123270 non-null  float64
 13  walks            123270 non-null  float64
 14  stolen_bases     123270 non-null  float64
 15  home/away        123270 non-null  object 
 16  singles          123270 non-null  floa

In [ ]:
X_win = win_data[['singles','doubles','triples','home_runs','walks','stolen_bases','sacrifice_flies']]
Y_win = win_data['score']

Xw_train, Xw_test, Yw_train, Yw_test = train_test_split(X_win,Y_win,test_size=0.2,random_state=33)
Xw_train = sm.add_constant(Xw_train)
Xw_test = sm.add_constant(Xw_test)
results1 = sm.OLS(Yw_train, Xw_train).fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                 2.918e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):               0.00
Time:                        15:30:24   Log-Likelihood:            -1.9585e+05
No. Observations:               98616   AIC:                         3.917e+05
Df Residuals:                   98608   BIC:                         3.918e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.7402      0.018   -1

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Next up I create a different regression, one using just total bases, walks, sacrifice flies, and stolen bases. This model performs slightly less well than the one where the types of hits are all taken into account. The R squared is slightly lower, as is the mean absolute error, so I will also leave this one alone.

In [ ]:
Xa = data[['total_bases','walks','sacrifice_flies','stolen_bases']]
Ya = data['score']

Xa_train, Xa_test, Ya_train, Ya_test = train_test_split(Xa, Ya, test_size=0.2, random_state=33)
Xa_train = sm.add_constant(Xa_train)
Xa_test = sm.add_constant(Xa_test)
results2 = sm.OLS(Ya_train, Xa_train).fit()
print(results2.summary())


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                 6.615e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):               0.00
Time:                        15:30:24   Log-Likelihood:            -1.8688e+05
No. Observations:               98616   AIC:                         3.738e+05
Df Residuals:                   98611   BIC:                         3.738e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.0314      0.014   -1

In [ ]:
lrm.fit(Xa_train,Ya_train)
yatrain_pred = lrm.predict(Xa_train)
yatest_pred = lrm.predict(Xa_test)
print('alternate training set R squared score is: {}'.format(lrm.score(Xa_train,Ya_train)))
print('alternate test set R squared score is: {}'.format(lrm.score(Xa_test,Ya_test)))
print('cross val score for alternate training set: {}'.format(cross_val_score(lrm,Xa_train,Ya_train,cv=5)))
print('mean squared error for test set: {}'.format(mean_squared_error(Ya_test,yatest_pred)))
print('mean absolute error for test set: {}'.format(mean_absolute_error(Ya_test,yatest_pred)))
print('median absolute error for test set: {}'.format(median_absolute_error(Ya_test,yatest_pred)))

alternate training set R squared score is: 0.7285105337447442
alternate test set R squared score is: 0.7247722640209879
cross val score for alternate training set: [0.72941531 0.72761018 0.73352019 0.72656351 0.72536774]
mean squared error for test set: 2.587668563515868
mean absolute error for test set: 1.241489666838941
median absolute error for test set: 1.0000343756119996


In [ ]:
data_plus = pd.read_csv('baseball_plus.csv')
data_plus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123270 entries, 0 to 123269
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       123270 non-null  int64  
 1   Unnamed: 0.1     123270 non-null  int64  
 2   year             123270 non-null  int64  
 3   team             123270 non-null  object 
 4   score            123270 non-null  int64  
 5   at_bats          123270 non-null  float64
 6   hits             123270 non-null  float64
 7   doubles          123270 non-null  float64
 8   triples          123270 non-null  float64
 9   home_runs        123270 non-null  float64
 10  rbi              123270 non-null  float64
 11  sacrifice_hits   123270 non-null  float64
 12  sacrifice_flies  123270 non-null  float64
 13  walks            123270 non-null  float64
 14  stolen_bases     123270 non-null  float64
 15  home/away        123270 non-null  object 
 16  singles          123270 non-null  floa

Here I'll add the label encoded team by year variable, as well as the label encoded ballpark variable. This does slightly improve the performance, however the difference seems nearly negligible.

In [ ]:
Xp = data_plus[['singles','doubles','triples','home_runs','walks','sacrifice_flies','stolen_bases',
                    'spec_team_enc','ballpark_enc']]
Yp = data_plus['score']
Xp_train, Xp_test, Yp_train, Yp_test = train_test_split(Xp,Yp,test_size=0.2,random_state=33)
Xp_train = sm.add_constant(Xp_train)
Xp_test = sm.add_constant(Xp_test)
results3 = sm.OLS(Yp_train, Xp_train).fit()
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                 3.040e+04
Date:                Thu, 04 Aug 2022   Prob (F-statistic):               0.00
Time:                        15:30:25   Log-Likelihood:            -1.8567e+05
No. Observations:               98616   AIC:                         3.714e+05
Df Residuals:                   98606   BIC:                         3.715e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -2.9605      0.069    -

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:

lrm.fit(Xp_train,Yp_train)
yptest_pred = lrm.predict(Xp_test)
yptrain_pred = lrm.predict(Xp_train)
print('R squared score for training set: {}'.format(lrm.score(Xp_train,Yp_train)))
print('R squared score for test set: {}'.format(lrm.score(Xp_train,Yp_train)))
print('cross validation scores: {}'.format(cross_val_score(lrm,Xp_train,Yp_train)))
print('mean squared error for the test set: {}'.format(mean_squared_error(Yp_test,yptest_pred)))
print('mean absolute error for the test set {}'.format(mean_absolute_error(Yp_test,yptest_pred)))
print('median absolute error for the test set {}'.format(median_absolute_error(Yp_test,yptest_pred)))

R squared score for training set: 0.7351080472237707
R squared score for test set: 0.7351080472237707
cross validation scores: [0.73645779 0.73421481 0.73944901 0.73302216 0.73214972]
mean squared error for the test set: 2.5239793722356407
mean absolute error for the test set 1.2312948030794657
median absolute error for the test set 0.999303216638497


To finish off I'll run a random forest with the data that has the two label encoded variables added. This is really just to see how much predictive power these variables have in terms of runs scored, the random forest algorithm is generally better at prediction than most others. However, despite the very high R squared score, the mean absolute error is slightly higher than in the regression, so the random forest isn't even the better option for simply predicting runs scored, given these specific features used.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
Xp = data_plus[['singles','doubles','triples','home_runs','walks','stolen_bases','sacrifice_flies',
                'ballpark_enc','spec_team_enc','year_enc']]
rfr = RandomForestRegressor()
rfr.fit(Xp_train,Yp_train)
rfr_pred = rfr.predict(Xp_test)
print('R squared score for test set: {}'.format(rfr.score(Xp_train,Yp_train)))


R squared score for test set: 0.9585308765239268


In [ ]:
print('mean squared error for random forest on test set: {}'.format(mean_squared_error(Yp_test,rfr_pred)))
print('mean absolute error for random forest on test set: {}'.format(mean_absolute_error(Yp_test,rfr_pred)))
print('median absolute error for random forest on test set: {}'.format(median_absolute_error(Yp_test,rfr_pred)))

mean squared error for random forest on test set: 2.774473714127517
mean absolute error for random forest on test set: 1.279674166013178
median absolute error for random forest on test set: 1.0199999999999996


Ultimately each regression here is trained on about 100,000 rows, and tested on about 20,000. In all cases with the linear regressions the R squared for the training set is about the same as the R squared for the test set, so it doesn't look like we have to worry about overfitting.

The first linear regression is a clear winner in my eyes here. The linear weights it gives for the different offensive outcomes are interesting. Singles are about 1.5 times more valuable than walks, and doubles about 1.5 times more valuable than singles. Home runs are about 3 times more valuable than singles, and triples are a little more than twice as valueable as singles. Stolen bases have a relatively low value (which isn't surprising seeing as they don't produce RBI's). And finally sacrifice flies are just slightly less valuable than doubles. Plugging in a players line score for each game (with these variables specifically) looks like a good way to measure the value of the player's production with respect to the potential for creating runs. Since there is already a statistic called runs created, I will not refer to this as runs created, in any case the runs created from Bill James does not use the individual hit types, but rather the total bases.

The advantage of this formula compared to one that uses total bases is we see that the total base values from different hits do not have the same ratios with each other as the values from this formula. That is, doubles are not quite twice as valuable as singles, and home runs are not twice as valuable as doubles and so on. Also this formula captures about 73-74% of the variation in runs scored, which isn't bad, given that runs scored is dependent on many factors, mainly situational ones. While some players do trend towards being clutch, e.g. hitting well with runners in scoring position and so on, things like that aren't as much in the player's control as offensive outcome aggregates. In the next notebook I will create a new statistic based on this formula, that also takes the year and the ballpark played in into consideration.